In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from tensorflow.keras.models import load_model

face_detector = YOLO("yolov8n-face.pt")
gender_model = load_model("gender_classification_model.h5")

video_path = "Videos/genequ1.mp4"
output_path = "output_video.mp4"

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = face_detector(frame, verbose=False)[0]
    male_count, female_count = 0, 0

    for box in results.boxes.xyxy.cpu().numpy():
        x1, y1, x2, y2 = map(int, box)
        face_roi = frame[y1:y2, x1:x2]

        if face_roi.shape[0] > 0 and face_roi.shape[1] > 0:
            face_resized = cv2.resize(face_roi, (64, 64)).astype(np.float32) / 255.0
            face_resized = np.expand_dims(face_resized, axis=0)

            prediction = gender_model.predict(face_resized, verbose=0)
            label = "Male" if np.argmax(prediction) == 0 else "Female"

            color = (255, 0, 0) if label == "Male" else (0, 0, 255)
            if label == "Male":
                male_count += 1
            else:
                female_count += 1

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.putText(frame, f"Male: {male_count} | Female: {female_count}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 
                0.9, (0, 255, 0), 2)

    out.write(frame)
    cv2.imshow("Multi-Face Gender Classification", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()